In [3]:
import pandas as pd
import sqlite3

In [4]:
df = pd.read_csv("weapons_incidents.csv", dtype = {'precinct':"str"})
df.dropna
df = df[df.precinct != "HP"]
df["incident_datetime"] = pd.to_datetime(df["incident_timestamp"])

In [5]:
df.precinct.unique()

array(['08', '05', '06', '09', '10', '02', '11', '03', '12', '07', '04',
       '0'], dtype=object)

In [6]:
df.drop(df.columns[[8,15]], axis = 1)

,X,Y,crime_id,report_number,address,offense_description,offense_category,state_offense_code,charge_description,incident_timestamp,...,precinct,block_id,neighborhood,council_district,zip_code,longitude,latitude,ibr_date,oid,incident_datetime
0,-83.226629,42.431652,3000108,1612060112,Cambridge Ave & Avon Ave,WEAPONS OFFENSE - OTHER,WEAPONS OFFENSES,5203,WEAPONS OFFENSE - OTHER,2016/12/07 08:40:00+00,...,08,261635407002006,O'Hair Park,1,48219,-83.226629,42.431652,2017/01/27 15:08:19+00,457,2016-12-07 08:40:00+00:00
1,-82.933384,42.414956,3000149,1612070029,Bluehill St & Berden St,WEAPONS OFFENSE - OTHER,WEAPONS OFFENSES,5203,WEAPONS OFFENSE - OTHER,2016/12/07 11:52:00+00,...,05,261635015005021,Cornerstone Village,4,48224,-82.933384,42.414956,2016/12/13 12:47:00+00,461,2016-12-07 11:52:00+00:00
2,-83.226076,42.354101,3000715,1612080166,Artesian St & Constance St,WEAPONS OFFENSE - CONCEALED,WEAPONS OFFENSES,5201,WEAPONS OFFENSE - CONCEALED,2016/12/08 22:15:00+00,...,06,261635460004006,Warrendale,7,48228,-83.226076,42.354101,2017/05/29 15:26:32+00,537,2016-12-08 22:15:00+00:00
3,-82.982927,42.377198,3001495,1612100258,Mack Ave & Lemay Ave,WEAPONS OFFENSE - CONCEALED,WEAPONS OFFENSES,5201,WEAPONS OFFENSE - CONCEALED,2016/12/11 07:22:00+00,...,05,261635141003008,West End,4,48214,-82.982927,42.377198,2017/01/27 15:08:19+00,636,2016-12-11 07:22:00+00:00
4,-82.946491,42.415965,3001508,1612110027,McKinney Ave & Kensington Ave,WEAPONS OFFENSE - CONCEALED,WEAPONS OFFENSES,5201,WEAPONS OFFENSE - CONCEALED,2016/12/11 11:55:00+00,...,09,261635010002007,Yorkshire Woods,4,48224,-82.946491,42.415965,2017/05/29 15:26:32+00,639,2016-12-11 11:55:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17503,-83.250619,42.400443,3831876,2211100246,Fenkell St & Westbrook St,WEAPONS OFFENSE - CONCEALED,WEAPONS OFFENSES,5201,WEAPONS OFFENSE - CONCEALED,2022/11/11 01:51:00+00,...,06,261635436002003,Brightmoor,1,48223,-83.250619,42.400443,NaN,482687,2022-11-11 01:51:00+00:00
17504,-83.194105,42.407088,3832222,2211110257,Pilgrim St & Coyle St,WEAPONS OFFENSE - OTHER,WEAPONS OFFENSES,5203,WEAPONS OFFENSE - OTHER,2022/11/12 01:45:00+00,...,02,261635375003001,Belmont,1,48227,-83.194105,42.407088,NaN,482750,2022-11-12 01:45:00+00:00
17505,-82.947808,42.425232,3832285,2211110253,Morang Dr & Lakepointe St,VIOLATION OF CONTROLED SUBSTANCE ACT - (VCSA),WEAPONS OFFENSES,5201,WEAPONS OFFENSE - CONCEALED,2022/11/12 01:21:00+00,...,09,261635009003000,Yorkshire Woods,4,48224,-82.947808,42.425232,NaN,482769,2022-11-12 01:21:00+00:00
17506,-82.944205,42.420898,3832711,2211130024,Balfour Rd & Britain St,WEAPONS OFFENSE - OTHER,WEAPONS OFFENSES,5203,WEAPONS OFFENSE - OTHER,2022/11/13 10:35:00+00,...,09,261635009002001,Yorkshire Woods,4,48224,-82.944205,42.420898,NaN,482856,2022-11-13 10:35:00+00:00


In [7]:
df.columns[:]

Index(['X', 'Y', 'crime_id', 'report_number', 'address', 'offense_description',
       'offense_category', 'state_offense_code', 'arrest_charge',
       'charge_description', 'incident_timestamp', 'incident_time',
       'day_of_week', 'hour_of_day', 'year', 'scout_car_area', 'precinct',
       'block_id', 'neighborhood', 'council_district', 'zip_code', 'longitude',
       'latitude', 'ibr_date', 'oid', 'incident_datetime'],
      dtype='object')

In [8]:
def month(row):
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October' , 'November', 'December']
    timestamp = row["incident_timestamp"]
    month = f"{str(timestamp[0:4])}-{str(timestamp[5:7])}"
    return month

In [9]:
def pre_post(row):
    year_month = row["incident_datetime"]
    if  year_month > pd.to_datetime('2020-03-15', utc=True) and year_month < pd.to_datetime("2021-03-15", utc=True):
        return "Pre"
    elif year_month > pd.to_datetime("2021-03-15", utc=True) and year_month < pd.to_datetime("2022-03-15", utc=True):
        return "Post"
    else: 
        return "Neither"
df["pre/post"] = df.apply(lambda row: pre_post(row), axis = 1)

In [10]:
def reassign(row):
    if row["precinct"] == '0':
        return "03"
    else:
        return row["precinct"]

df["precinct"] = df.apply(lambda row: reassign(row), axis = 1)

In [11]:
df["year_month"] = df.apply(lambda row: month(row), axis = 1)

In [12]:
df_gb = df.groupby(["year", "year_month", "precinct", "offense_category"])
gb_df = df_gb.size().reset_index()

In [13]:
gb_df["offense_category"].unique()

array(['WEAPONS OFFENSES'], dtype=object)

In [14]:
def crime_bucket(row):
    bucket_dict = {"Property Damage" : ["ARSON", "DAMAGE TO PROPERTY"], 
    'Civil Disturbance' : ["FAMILY OFFENSE", "DISORDERLY CONDUCT"],
    'Nonfatal Violence' : ["AGGRAVATED ASSAULT", "ASSAULT"],
    'Fatal Violence' : ["JUSTIFIABLE HOMICIDE", "HOMICIDE"], 
    'Controlled Substances' : ["DANGEROUS DRUGS", "OUIL", "LIQUOR"],
    'Weapons' : ["WEAPONS OFFENSES"],
    'Sexual Violence' : ["SEX OFFENSES", "SEXUAL ASSAULT"],
    'Property Theft' : ["LARCENY", "BURGLARY", "STOLEN PROPERTY", "ROBBERY"]}
    row_cat = row["offense_category"]
    category = "Other"
    for key, value in bucket_dict.items():
        if row_cat in value:
            category = key
            break
        else: 
            continue
    return category

In [15]:
gb_df["crime_bucket"] = gb_df.apply(lambda row: crime_bucket(row), axis = 1)

In [16]:
gb_df.columns.values[4] = 'counts'
gb_df

,year,year_month,precinct,offense_category,counts,crime_bucket
0,1978,1978-04,08,WEAPONS OFFENSES,1,Weapons
1,1979,1979-06,10,WEAPONS OFFENSES,1,Weapons
2,2000,2000-07,04,WEAPONS OFFENSES,1,Weapons
3,2001,2001-09,02,WEAPONS OFFENSES,1,Weapons
4,2002,2002-07,10,WEAPONS OFFENSES,1,Weapons
...,...,...,...,...,...,...
801,2022,2022-11,08,WEAPONS OFFENSES,26,Weapons
802,2022,2022-11,09,WEAPONS OFFENSES,23,Weapons
803,2022,2022-11,10,WEAPONS OFFENSES,15,Weapons
804,2022,2022-11,11,WEAPONS OFFENSES,6,Weapons


In [17]:
gb_df.to_csv("crimes_byPCT.csv", index = False)

In [26]:
weap_df = gb_df[gb_df.crime_bucket == "Weapons"]
weap_df = weap_df[weap_df["year"] > 2014]
weap_df[weap_df["year"] < 2014]

,year,year_month,precinct,offense_category,counts,crime_bucket


In [23]:
weap_df.to_csv("weapon_crimes_byPCT.csv", index = False)

In [18]:
by_period_df = gb_df
by_period_df["year_month_datetime"] = pd.to_datetime(gb_df["year_month"])
by_period_df

,year,year_month,precinct,offense_category,counts,crime_bucket,year_month_datetime
0,1978,1978-04,08,WEAPONS OFFENSES,1,Weapons,1978-04-01
1,1979,1979-06,10,WEAPONS OFFENSES,1,Weapons,1979-06-01
2,2000,2000-07,04,WEAPONS OFFENSES,1,Weapons,2000-07-01
3,2001,2001-09,02,WEAPONS OFFENSES,1,Weapons,2001-09-01
4,2002,2002-07,10,WEAPONS OFFENSES,1,Weapons,2002-07-01
...,...,...,...,...,...,...,...
801,2022,2022-11,08,WEAPONS OFFENSES,26,Weapons,2022-11-01
802,2022,2022-11,09,WEAPONS OFFENSES,23,Weapons,2022-11-01
803,2022,2022-11,10,WEAPONS OFFENSES,15,Weapons,2022-11-01
804,2022,2022-11,11,WEAPONS OFFENSES,6,Weapons,2022-11-01


In [19]:
df_gb_period = df[(df["offense_category"] == "WEAPONS OFFENSES") & (df["pre/post"] != "Neither")].groupby(["precinct", "pre/post"]) 
gb_period_df = df_gb_period.size().reset_index()
gb_period_df.columns.values[2] = 'counts'
gb_period_df.to_csv("weapon_crimes_periods_byPCT.csv", index = False)
gb_period_df.head()

,precinct,pre/post,counts
0,02,Post,270
1,02,Pre,478
2,03,Post,382
3,03,Pre,408
4,04,Post,326


In [20]:
byperiod_pvt = pd.pivot(gb_period_df, columns = 'pre/post', values = 'counts', index = 'precinct')
byperiod_pvt.head()

pre/post,Post,Pre
precinct,,
02,270,478
03,382,408
04,326,235
05,357,388
06,491,530


In [21]:
def pct_change(row):
    pre = row["Pre"]
    post = row["Post"]
    return ((post - pre) / pre) * 100

byperiod_pvt["pct_change"] = byperiod_pvt.apply(lambda row: pct_change(row), axis = 1)
byperiod_pvt.to_csv("weapons_pct_change.csv")

In [22]:
byperiod_pvt.head()

pre/post,Post,Pre,pct_change
precinct,,,
02,270,478,-43.514644
03,382,408,-6.372549
04,326,235,38.723404
05,357,388,-7.989691
06,491,530,-7.358491


In [24]:
weapons_assaults_df = df[df["offense_description"].str.contains("AGGRAVATED")]
weapons_df_gb_period = weapons_assaults_df[(weapons_assaults_df["offense_category"] == "WEAPONS OFFENSES") & (weapons_assaults_df["pre/post"] != "Neither")].groupby(["precinct", "pre/post"]) 
gb_weapons_period_df = weapons_df_gb_period.size().reset_index()
gb_weapons_period_df.columns.values[2] = 'counts'
assault_byperiod_pvt = pd.pivot(gb_weapons_period_df, columns = 'pre/post', values = 'counts', index = 'precinct')
assault_byperiod_pvt["pct_change"] = assault_byperiod_pvt.apply(lambda row: pct_change(row), axis = 1)
assault_byperiod_pvt
assault_byperiod_pvt.to_csv("assault_pct_change.csv")